In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers

In [2]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

model_path="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer=AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
%%time

def generate_essay(prompt):
    messages = [{
        "role":"user",
        "content": prompt
    }]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors = "pt").to('cuda')
    
    # Setting `pad_token_id` to `eos_token_id` for open-ended generation.
    generated_ids = model.generate(
        model_inputs,
        max_new_tokens = 7500,
        do_sample = True,
        pad_token_id = tokenizer.eos_token_id
        
    )

    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    text = decoded[0].split("[/INST]")[1]
    return text

prompt = '''
Obituary Details: 
Yulonda “Lonnie” Bascia Hollis, 49, of Packard Ave., Kannapolis, N.C., passed away on Tuesday, August 31, 2021 at her residence.  She was born August 18, 1972 in Cabarrus County to Kenneth Crawford and Lorene Hollis Russell. Yulonda was educated in the Cabarrus County School System  and attended A.L. Brown High School.  She was formerly employed with FedEx.  Services will be held on Wednesday, September 8, 2021, at Living Water Church of God 166 N. Little Texas Rd. Kannapolis.  Visitation will be 1:00 PM – 2:00 PM; with funeral services at 2:00 PM. Burial will follow at Bethel Baptist Church Cemetery. She was preceded in death by a brother, Cedric Hill; grandmother, Essie Hollis and stepfathers, Robin Brown  and Cleatus Gill. She leaves to cherish fond memories two daughters, Kiana Hollis and Jasmine Hollis both of Kannapolis; one son, Clinton Thompson of Kannapolis; seven grandchildren; her parents, Kenneth Crawford (Regina) pf Mooresville, NC, Lorene Russell of Kannapolis;  fiancé, Len Harold; one brother, Rodricko Hill (Lavita) of Charlotte, NC; four sisters, Gytanna Hollis of Concord, Shakitta Hill (Jermaine Smith) of Kannapolis, Tara Crawford of Mooresville, NC, Melonie Patrice Love of Dallas, TX; grandmother, Lula Mae Brown of Charlotte, NC; grandfather, Cletus Gill (Sue) of Kannapolis; stepbrother, Rev. Chavis Maxwell (Tosha) of Kannapolis; stepsister, Tameca Robinson of Salisbury and a host of nieces, nephews, cousins and friends.

Task:
Extract  below attributes if exists from the above obituary details.
name, age, death city, death state, death date, birth date, birth city, parents, grandparents, children ,spouse, sons_or_daughters_in_law, college, occupations, funeral home name

Output Format: Json object
'''


CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 8.11 µs


In [4]:
text = generate_essay(prompt)
text

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


' {\n"name": "Yulonda Bascia Hollis",\n"age": 49,\n"death city": "Kannapolis",\n"death state": "N.C.",\n"death date": "August 31, 2021",\n"birth date": "August 18, 1972",\n"birth city": "Cabarrus County",\n"parents": {\n"father": "Kenneth Crawford",\n"mother": "Lorene Hollis Russell"\n},\n"grandparents": {\n"maternal": "Essie Hollis",\n"paternal": "Cletus Gill"\n},\n"children": [\n{\n"name": "Kiana Hollis",\n"relationship": "daughter"\n},\n{\n"name": "Jasmine Hollis",\n"relationship": "daughter"\n},\n{\n"name": "Clinton Thompson",\n"relationship": "son"\n}\n],\n"spouse": {\n"name": "fiancé",\n"relationship": "Len Harold"\n}\n}'